# 读取数据

In [386]:
import pandas as pd
import numpy as np

In [387]:
order = pd.read_table("training_data/order_data/order_data_2016-01-01", names = ["order_id", "driver_id", "passenger_id", "start_district_hash", "dest_district_hash", "price", "time"])
cluster = pd.read_table("training_data/cluster_map/cluster_map", names = ["district_hash", "district_id"])
weather = pd.read_table("training_data/weather_data/weather_data_2016-01-01", names = ["time", "weather", "temperature", "PM2.5"])
traffic = pd.read_table("training_data/traffic_data/traffic_data_2016-01-01", names = ["district_hash","level_1", "level_2", "level_3", "level_4", "tj_time"])

# 数据整理

#### 订单数据去重

In [388]:
order = order.drop_duplicates() 

In [389]:
order.shape

(498824, 7)

#### 计算order时间片字段

In [390]:
time = pd.DataFrame((x.split(' ') for x in order['time']), columns=['date','t'])
s = pd.DataFrame((x.split(':') for x in time['t']), columns=['hour','minute','second'])

In [391]:
s["hour"] = s["hour"].astype(int)
s["minute"] = s["minute"].astype(int)
s["second"] = s["second"].astype(int)

In [392]:
s.head()

,hour,minute,second
0,13,37,23
1,9,47,54
2,18,24,2
3,22,13,27
4,17,0,6


In [393]:
s["value"] = s["hour"]*6 + s["minute"]//10 + 1
order["time_slient"] = s["value"]

In [394]:
order = order.drop("time", axis=1)

#### 计算weather时间片字段

In [395]:
weather_time = pd.DataFrame((x.split(' ') for x in weather['time']), columns=['date','t'])
t = pd.DataFrame((x.split(':') for x in weather_time['t']), columns=['hour','minute','second'])

In [396]:
t["hour"] = t["hour"].astype(int)
t["minute"] = t["minute"].astype(int)
t["second"] = t["second"].astype(int)

In [397]:
t["value"] = t["hour"]*6 + t["minute"]//10 + 1
weather["time_slient"] = t["value"]

#### 处理traffic数据

In [398]:
le1 = pd.DataFrame((x.split(':') for x in traffic['level_1']), columns=['level','num'])
le2 = pd.DataFrame((x.split(':') for x in traffic['level_2']), columns=['level','num'])
le3 = pd.DataFrame((x.split(':') for x in traffic['level_3']), columns=['level','num'])
le4 = pd.DataFrame((x.split(':') for x in traffic['level_4']), columns=['level','num'])

In [399]:
traffic['level_1'] = le1['num']
traffic['level_2'] = le2['num']
traffic['level_3'] = le3['num']
traffic['level_4'] = le4['num']

In [400]:
traffic_time = pd.DataFrame((x.split(' ') for x in traffic['tj_time']), columns=['date','t'])
tr = pd.DataFrame((x.split(':') for x in traffic_time['t']), columns=['hour','minute','second'])

In [401]:
tr["hour"] = tr["hour"].astype(int)
tr["minute"] = tr["minute"].astype(int)
tr["second"] = tr["second"].astype(int)

In [402]:
tr["value"] = tr["hour"]*6 + tr["minute"]//10 + 1
traffic["time_slient"] = tr["value"]

#### 与cluster数据集合并操作

In [403]:
order = pd.merge(order, cluster, how = 'left', left_on="start_district_hash", right_on="district_hash")

In [404]:
order.head()

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,price,time_slient,district_hash,district_id
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,82.0,4725c39a5e5f4c188d382da3910b3f3f,23
1,92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,59.0,82cc4851f9e4faa4e54309f8bb73fd7c,8
2,abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,111.0,fff4e8465d1e12621bc361276b6217cf,32
3,cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11.0,134.0,4b7f6f4e2bf237b6cc58f57142bea5c0,13
4,139d492189ae5a933122c098f63252b3,NaN,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4.0,103.0,fc34648599753c9e74ab238e9a4a07ad,27


#### 计算每个区域对应的需求和缺口

In [405]:
grouped_driver = order["driver_id"].groupby(order["time_slient"]).count()
grouped_order = order["order_id"].groupby(order["time_slient"]).count()

In [406]:
o = dict(grouped_order)
d = dict(grouped_driver)

In [407]:
order["total"] = order["time_slient"].replace(o) 
order["completed"] = order["time_slient"].replace(d)

In [408]:
order["gap"] = order["total"] - order["completed"]
order["gap_percent"] = order["gap"] / order["total"]

In [409]:
pd.set_option('precision',2)

In [410]:
order["driver_is_null"] = pd.isnull(order["driver_id"])

####  删除冗余的数据

In [413]:
clean_order = order.drop('order_id', axis=1)
clean_order = clean_order.drop('driver_id', axis=1)
clean_order = clean_order.drop('start_district_hash', axis=1)
clean_order = clean_order.drop('district_hash', axis=1)
clean_order = clean_order.drop("completed", axis=1)
clean_order = clean_order.drop("passenger_id", axis=1)
clean_order = clean_order.drop('dest_district_hash', axis=1)
clean_order = clean_order.drop('dest_district_hash', axis=1)
clean_order = clean_order.drop('dest_district_hash', axis=1)
clean_order = clean_order.drop('dest_district_hash', axis=1)

#### 导出csv文件

In [418]:
clean_order.to_csv('cleaned_order_01-01.csv')

#### 与天气和交通数据合并

In [311]:
order = pd.merge(order, weather, how = 'left', on="time_slient")

In [312]:
order = pd.merge(order, traffic, on=["time_slient","district_hash"])

#### 导出合并后的csv文件

In [ ]:
order.to_csv('order_01-01.csv')

In [417]:
clean_order.head()

,price,time_slient,district_id,driver_is_null
0,24.0,82.0,23,False
1,2.0,59.0,8,False
2,9.0,111.0,32,False
3,11.0,134.0,13,False
4,4.0,103.0,27,True


In [416]:
clean_order = clean_order.drop('total', axis=1)
clean_order = clean_order.drop('gap', axis=1)
clean_order = clean_order.drop('gap_percent', axis=1)